In [4]:
import numpy as np
from fxpmath import Fxp

In [38]:
class myfi(np.ndarray):
    def __new__(cls, input_array, signed=1, w=32, f=16, like=None, quantize=True, **kwargs): 
        ndarray = np.asarray(input_array).view(cls)
        if isinstance(like, myfi):
            ndarray.config(like.kwargs)
            ndarray.resize(like.signed, like.w, like.f, quantize)            
        else:     
            ndarray.config(kwargs)                       
            ndarray.resize(signed, w, f, quantize)            
        return ndarray

    def resize(self, signed, w, f, quantize=True):
        self.signed = signed
        self.w = w
        self.f = f
        self.i = w-f
        if self.w < self.f:
            raise ValueError("fraction length > word length is not support")
        self.precision = 2**-self.f
        upper = 2**(self.w - self.f - (1 if self.signed else 0))
        self.upper = upper - self.precision 
        self.lower = -2**(self.w-self.f-1) if self.signed else 0    
        
        if quantize:
            self.quantize() 
        return self

    def quantize(self):
        print(f'Q: {self.size}')
        b = self.ndarray * (2**self.f)
        if self.rounding == 'nearest':
            store_int = np.round(b)
        elif self.rounding == 'floor':
            store_int = np.floor(b)
        else:
            raise ValueError(f'not support rounding method {self.rounding}')
        # overflow/underflow
        if self.overflow == 'warp':
            m = (1 << self.w)
            if signed: 
                store_int &= (m - 1) 
                store_int[store_int>(1 << (self.w-1))] |= (-m)
            else: 
                store_int &= (m - 1) 
            self[...] = store_int*self.precision # this will only reassgin array value, instead of create new myfi instance
        elif self.overflow == 'saturate':
            self[...] = store_int*self.precision
            self[self>self.upper] = self.upper  # np.clip(self,self.upper,self.lower,out=self) will do the same job, but slower
            self[self<self.lower] = self.lower

    def config(self, kwargs):
        self.rounding = kwargs.get('rounding', 'nearest')
        self.overflow = kwargs.get('overflow', 'warp')
        self.fixed = kwargs.get('fixed', False)

    @property
    def ndarray(self):
        return self.view(np.ndarray)
    @property
    def int(self):
        return (self.ndarray*2**self.f).astype(int) # return normal ndarray
    @property
    def kwargs(self):
        return {
            'rounding' : self.rounding,
            'overflow' : self.overflow,
            'fixed': self.fixed
        }

    def base_repr(self, base=2, frac_point=False): # return ndarray with same shape and dtype = '<Uw' where w=self.w
        if base == 2:
            add_point = lambda s: s[:-self.f] + '.' + s[-self.f:] if frac_point else s
            func = lambda i: add_point(np.binary_repr(i,width=self.w))    
        else:
            func = lambda i: np.base_repr(i,base=base)
        return np.vectorize(func)(self.int)
    @property
    def bin(self): 
        return self.base_repr(2)
    @property 
    def bin_(self):
        return self.base_repr(2,frac_point=True)
    @property
    def hex(self):
        return self.base_repr(16)

    # overload operands    
    def __repr__(self):
        signed = 's' if self.signed else 'u'
        return f'myfi-{signed}{self.w}/{self.f}: \n' + self.ndarray.__repr__() 
    __str__ = __repr__
    def __getitem__(self, key):
        # let single element index of array return myfi with shape (1,) instead of base dtype element
        return myfi(super().__getitem__(key), like=self)
    def __neg__(self):
        return myfi(super().__neg__(), like=self)
    def __pos__(self):
        return self    
    def __arithmeticADD__(self, func, y):
        y = y if isinstance(y, myfi) else myfi(y, like=self)
        i = max(self.i, y.i)
        f = max(self.f, y.f)
        return myfi(func(y.ndarray), self.signed|y.signed, i+f+1, f, None, False, **self.kwargs)         
    def __arithmeticMUL__(self, func, y):
        y = y if isinstance(y, myfi) else myfi(y, like=self)
        return myfi(func(y.ndarray), self.signed|y.signed, self.w+y.w, self.f+y.f, None, False, **self.kwargs) 
    # arithmetic
    __add__         = lambda self,y: self.__arithmeticADD__(super().__add__, y)
    __radd__        = lambda self,y: self.__arithmeticADD__(super().__radd__, y)
    __iadd__        = lambda self,y: self.__arithmeticADD__(super().__iadd__, y)
    __sub__         = lambda self,y: self.__arithmeticADD__(super().__sub__, y)
    __rsub__        = lambda self,y: self.__arithmeticADD__(super().__rsub__, y)
    __isub__        = lambda self,y: self.__arithmeticADD__(super().__isub__, y)
    __mul__         = lambda self,y: self.__arithmeticMUL__(super().__mul__, y)
    __rmul__        = lambda self,y: self.__arithmeticMUL__(super().__rmul__, y)
    __imul__        = lambda self,y: self.__arithmeticMUL__(super().__imul__, y)
    __truediv__     = lambda self,y: self.__arithmeticMUL__(super().__truediv__, y)
    __rtruediv__    = lambda self,y: self.__arithmeticMUL__(super().__rtruediv__, y)
    __itruediv__    = lambda self,y: self.__arithmeticMUL__(super().__itruediv__, y)
    __floordiv__    = lambda self,y: self.__arithmeticMUL__(super().__floordiv__, y)
    __rfloordiv__   = lambda self,y: self.__arithmeticMUL__(super().__rfloordiv__, y)
    __ifloordiv__   = lambda self,y: self.__arithmeticMUL__(super().__ifloordiv__, y)
    #TODO: more advance operation? (shift expand/quantize of logical operation)
    __mod__         = lambda self,y: myfi(super().__mod__(y.view(np.ndarray)), like=self)
    __lshift__      = lambda self,y: myfi(super().__lshift__(y.view(np.ndarray)), like=self)
    __rshift__      = lambda self,y: myfi(super().__rshift__(y.view(np.ndarray)), like=self)
    __and__         = lambda self,y: myfi(super().__and__(y.view(np.ndarray)), like=self) 
    __or__          = lambda self,y: myfi(super().__or__(y.view(np.ndarray)), like=self) 
    __xor__         = lambda self,y: myfi(super().__xor__(y.view(np.ndarray)), like=self) 
    __invert__      = lambda self,y: myfi(super().__invert__(y.view(np.ndarray)), like=self)
    # comparison
    __eq__ = lambda self,y: self.ndarray == y
    __ne__ = lambda self,y: self.ndarray != y
    __ge__ = lambda self,y: self.ndarray >= y
    __gt__ = lambda self,y: self.ndarray >  y
    __le__ = lambda self,y: self.ndarray <= y
    __lt__ = lambda self,y: self.ndarray <  y
    #TODO: casting


x = np.random.rand(3,2,2)
y = myfi(x,1,14,3) # call __new__ and __array_finalize__
print(y.w,y.f,y.precision)
z = y[1:]
print(z.w,z.f,z.precision)
q = myfi(x,like=y)
print(q.w,q.f,q.precision)
w = y[1,1,1]
print(w.w,w.f,w.precision)
# y[1,1,1] # call __array_finalize__
# x.view(myfi) # call __array_fianlize__

Q: 12
14 3 0.125
Q: 8
14 3 0.125
Q: 12
14 3 0.125
Q: 1
14 3 0.125


In [34]:
s = 1
w = 15
f = 8
x = np.linspace(0,100,77)
print('init fx')
fx = myfi(x,s,w,f)
y = 1/3
print('init fy')
fy = myfi(y,s,w,f+2)
xy = x + y
print('fx+fy')
fxy = fx + fy
# xyf = myfi(fx.view(np.ndarray)+y, s,w+1,f)
# print(fxy == xyf)
# print(fxy.ndarray)

init fx
Q: 77
init fy
Q: 1
fx+fy


In [44]:
x = myfi(np.linspace(0,20,100),1,8,4,overflow='saturate')
x

Q: 100


myfi-s8/4: 
array([0.    , 0.1875, 0.375 , 0.625 , 0.8125, 1.    , 1.1875, 1.4375,
       1.625 , 1.8125, 2.    , 2.25  , 2.4375, 2.625 , 2.8125, 3.    ,
       3.25  , 3.4375, 3.625 , 3.8125, 4.0625, 4.25  , 4.4375, 4.625 ,
       4.875 , 5.0625, 5.25  , 5.4375, 5.6875, 5.875 , 6.0625, 6.25  ,
       6.4375, 6.6875, 6.875 , 7.0625, 7.25  , 7.5   , 7.6875, 7.875 ,
       7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375,
       7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375,
       7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375,
       7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375,
       7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375,
       7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375,
       7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375, 7.9375,
       7.9375, 7.9375, 7.9375, 7.9375])

In [57]:
def wrap(x, signed, n_word): 
    m = (1 << n_word)
    if signed: 
        x = np.array(x) & (m - 1) 
        x = np.where(x < (1 << (n_word-1)), x, x | (-m)) 
    else: 
        x = np.array(x) & (m - 1) 
    return x
def wrap2(x, signed, n_word):
    m = (1 << n_word)
    if signed: 
        x &= (m - 1) 
        x[x>(1 << (n_word-1))] |= (-m)
    else: 
        x &= (m - 1) 
    return xx


In [61]:
x = np.arange(-150,150, dtype=int)
# %timeit wrap(x,1,8)
# %timeit wrap2(x,1,8)

In [64]:
m = (1 << n_word)
upper = (1<<(n_word-1)) - 1 
lower = -1<<n_word-1
xx = x.copy()
xx[xx>upper] &= m-1
xx

array([-150, -149, -148, -147, -146, -145, -144, -143, -142, -141, -140,
       -139, -138, -137, -136, -135, -134, -133, -132, -131, -130, -129,
       -128, -127, -126, -125, -124, -123, -122, -121, -120, -119, -118,
       -117, -116, -115, -114, -113, -112, -111, -110, -109, -108, -107,
       -106, -105, -104, -103, -102, -101, -100,  -99,  -98,  -97,  -96,
        -95,  -94,  -93,  -92,  -91,  -90,  -89,  -88,  -87,  -86,  -85,
        -84,  -83,  -82,  -81,  -80,  -79,  -78,  -77,  -76,  -75,  -74,
        -73,  -72,  -71,  -70,  -69,  -68,  -67,  -66,  -65,  -64,  -63,
        -62,  -61,  -60,  -59,  -58,  -57,  -56,  -55,  -54,  -53,  -52,
        -51,  -50,  -49,  -48,  -47,  -46,  -45,  -44,  -43,  -42,  -41,
        -40,  -39,  -38,  -37,  -36,  -35,  -34,  -33,  -32,  -31,  -30,
        -29,  -28,  -27,  -26,  -25,  -24,  -23,  -22,  -21,  -20,  -19,
        -18,  -17,  -16,  -15,  -14,  -13,  -12,  -11,  -10,   -9,   -8,
         -7,   -6,   -5,   -4,   -3,   -2,   -1,   

In [67]:
np.int8(x)

array([ 106,  107,  108,  109,  110,  111,  112,  113,  114,  115,  116,
        117,  118,  119,  120,  121,  122,  123,  124,  125,  126,  127,
       -128, -127, -126, -125, -124, -123, -122, -121, -120, -119, -118,
       -117, -116, -115, -114, -113, -112, -111, -110, -109, -108, -107,
       -106, -105, -104, -103, -102, -101, -100,  -99,  -98,  -97,  -96,
        -95,  -94,  -93,  -92,  -91,  -90,  -89,  -88,  -87,  -86,  -85,
        -84,  -83,  -82,  -81,  -80,  -79,  -78,  -77,  -76,  -75,  -74,
        -73,  -72,  -71,  -70,  -69,  -68,  -67,  -66,  -65,  -64,  -63,
        -62,  -61,  -60,  -59,  -58,  -57,  -56,  -55,  -54,  -53,  -52,
        -51,  -50,  -49,  -48,  -47,  -46,  -45,  -44,  -43,  -42,  -41,
        -40,  -39,  -38,  -37,  -36,  -35,  -34,  -33,  -32,  -31,  -30,
        -29,  -28,  -27,  -26,  -25,  -24,  -23,  -22,  -21,  -20,  -19,
        -18,  -17,  -16,  -15,  -14,  -13,  -12,  -11,  -10,   -9,   -8,
         -7,   -6,   -5,   -4,   -3,   -2,   -1,   

In [69]:
x & -1

array([-150, -149, -148, -147, -146, -145, -144, -143, -142, -141, -140,
       -139, -138, -137, -136, -135, -134, -133, -132, -131, -130, -129,
       -128, -127, -126, -125, -124, -123, -122, -121, -120, -119, -118,
       -117, -116, -115, -114, -113, -112, -111, -110, -109, -108, -107,
       -106, -105, -104, -103, -102, -101, -100,  -99,  -98,  -97,  -96,
        -95,  -94,  -93,  -92,  -91,  -90,  -89,  -88,  -87,  -86,  -85,
        -84,  -83,  -82,  -81,  -80,  -79,  -78,  -77,  -76,  -75,  -74,
        -73,  -72,  -71,  -70,  -69,  -68,  -67,  -66,  -65,  -64,  -63,
        -62,  -61,  -60,  -59,  -58,  -57,  -56,  -55,  -54,  -53,  -52,
        -51,  -50,  -49,  -48,  -47,  -46,  -45,  -44,  -43,  -42,  -41,
        -40,  -39,  -38,  -37,  -36,  -35,  -34,  -33,  -32,  -31,  -30,
        -29,  -28,  -27,  -26,  -25,  -24,  -23,  -22,  -21,  -20,  -19,
        -18,  -17,  -16,  -15,  -14,  -13,  -12,  -11,  -10,   -9,   -8,
         -7,   -6,   -5,   -4,   -3,   -2,   -1,   

In [68]:
(m-1) | (-m)

-1